In [1]:
import os

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import configparser


models = np.array([
                   'MRM61',
                   'ESRA',
                   'Heliosat1I',
                   'SOLISsimple',
                   'CEM',
                   'MMAC',
                   'METSTAT',
                  ])


models_label = np.array([
                         'MRMv6.1',
                         'ESRA',
                         'Heliosat-1',
                         'Solis simple',
                         'CEM',
                         'MMAC',
                         'METSTAT',
 
                        ])

models_aodwvl = np.array([
                           550,
                           550,
                           550,
                           700,
                           0,
                           0,
                           0,
                          ])

### load dataset
# This dataset is produced with make_dataset_AOD_CSM_TCARS.py
config = configparser.ConfigParser()
config.read("ConfigFile.ini")
pf = config['PATHS']['datasets']

fname_CAMS_AOD = os.path.join(pf,"TCARS_CSF_AOD_2015.nc")
dsAOD = xr.open_dataset(fname_CAMS_AOD)

In [2]:
def calc_AE(t1,t2,l1,l2):
    """ calculate Angstrom exponent from aods (t1,t2)
        at two different wavelengths (l1,l2)
    """
    return -np.log(t1/t2)/np.log(l1/l2)
def calc_AOD(l1,t2,l2,ae):
    return t2*(l1/l2)**(-ae)

## Make Table 08
Compare clear sky fittet AOD from the CSM to T-CARS AOD

In [3]:
def get_metrics(O,P):
    #ensure no nans in data
    idx = ~np.isnan(O)*~np.isinf(O)
    idx*= ~np.isnan(P)*~np.isinf(P)
    O = O[idx]
    P = P[idx]
    # precalculation
    N = len(O)
    N1 = 1./float(N)
    N2 = 1./(float(N)+1.)
    DELTA = P-O
    SUM = P+O
    # MBE
    MBE = N1 * np.sum(DELTA)
    # RMSE
    RMSE = np.sqrt(N2*np.sum(DELTA**2))
    # R
    ccoef = np.corrcoef(P,O)
    ccoef = ccoef[0][1]
    return MBE,RMSE,ccoef

AODvgl=dsAOD.copy()
AODvgl = AODvgl.where(AODvgl.CAOD>0,drop=True)
AODvgl = AODvgl.where(AODvgl.CAOD<0.7,drop=True)
AODvgl = AODvgl.where(AODvgl.CAOD.round(3)!=0.3,drop=True)


## make Table
print("\\multicolumn{5}{|c|}{daily average AOD at 550\\,\\unit{nm}}}") 
print("\\middlehline")
# for wvl in [0,550,700]:
#     idx = models_aodwvl==wvl
    

#     for i,model in enumerate(zip(models[idx],models_label[idx])):
for m,model in enumerate(zip(models,models_label)):
    wvl = models_aodwvl[m]
    CAODs=[]
    model,mlabel=model

    AOD = AODvgl.mean(dim=['station'],skipna=True).sel(model=model)
    if wvl == 0: 
        AOD.CAOD.values =(AOD.CAOD * (AOD.AOD550 / AOD.AOD0)).values
    if wvl == 700:
        AE = calc_AE(AOD.AOD550,AOD.AOD700,550,700)
        AOD.CAOD.values = calc_AOD(550.,AOD.CAOD.values,700,AE.values)#(AOD.CAOD * (AOD.AOD550 / AOD.AOD700)).values
    AODseas = AOD.groupby('day.season').mean(skipna=True)
    for seas in ['DJF','MAM','JJA','SON']:
        CAODs.append(AODseas.CAOD.sel(season=seas).values)
    CAODs.append(AOD.CAOD.mean(dim='day',skipna=True).values)
    EARE = AOD[f'AOD550'].values
    DARE = AOD.CAOD.values
    mbe,rmse,corr = get_metrics(EARE,DARE)   

    print(f"{mlabel:12s}  & {CAODs[4]:6.2f} & {mbe:6.2f} & {rmse:6.2f} & {corr:6.2f} \\\\")

        

print("\\bottomhline")


\multicolumn{5}{|c|}{daily average AOD at 550\,\unit{nm}}}
\middlehline
MRMv6.1       &   0.10 &  -0.04 &   0.08 &   0.50 \\
ESRA          &   0.12 &  -0.01 &   0.06 &   0.68 \\
Heliosat-1    &   0.07 &  -0.05 &   0.08 &   0.63 \\
Solis simple  &   0.10 &  -0.03 &   0.08 &   0.48 \\
CEM           &   0.15 &   0.01 &   0.06 &   0.55 \\
MMAC          &   0.38 &   0.25 &   0.29 &   0.71 \\
METSTAT       &   0.13 &  -0.01 &   0.06 &   0.74 \\
\bottomhline


In [4]:
## same Tab08 but AOD not scaled
def get_metrics(O,P):
    #ensure no nans in data
    idx = ~np.isnan(O)*~np.isinf(O)
    idx*= ~np.isnan(P)*~np.isinf(P)
    O = O[idx]
    P = P[idx]
    # precalculation
    N = len(O)
    N1 = 1./float(N)
    N2 = 1./(float(N)+1.)
    DELTA = P-O
    SUM = P+O
    # MBE
    MBE = N1 * np.sum(DELTA)
    # RMSE
    RMSE = np.sqrt(N2*np.sum(DELTA**2))
    # R
    ccoef = np.corrcoef(P,O)
    ccoef = ccoef[0][1]
    return MBE,RMSE,ccoef

AODvgl=dsAOD.copy()
AODvgl = AODvgl.where(AODvgl.CAOD>0,drop=True)
AODvgl = AODvgl.where(AODvgl.CAOD<0.7,drop=True)
AODvgl = AODvgl.where(AODvgl.CAOD.round(3)!=0.3,drop=True)


## make Table
for wvl in [0,550,700]:
    idx = models_aodwvl==wvl
    print("\\multirow{%d}{*}{\\rotatebox[origin=c]{90}{%d\\,\\unit{nm}}}"%(1+np.count_nonzero(idx),wvl)) 

    for i,model in enumerate(zip(models[idx],models_label[idx])):
        CAODs=[]
        model,mlabel=model
        
        AOD = AODvgl.mean(dim=['station'],skipna=True).sel(model=model)
        AODseas = AOD.groupby('day.season').mean(skipna=True)
        for seas in ['DJF','MAM','JJA','SON']:
            CAODs.append(AODseas.CAOD.sel(season=seas).values)
        CAODs.append(AOD.CAOD.mean(dim='day',skipna=True).values)
        EARE = AOD[f'AOD{wvl:d}'].values
        DARE = AOD.CAOD.values
        mbe,rmse,corr = get_metrics(EARE,DARE)   

        print(f"& {mlabel:12s}  & {CAODs[0]:6.2f} & {CAODs[1]:6.2f} & {CAODs[2]:6.2f}  & {CAODs[3]:6.2f}  & {CAODs[4]:6.2f} & {mbe:6.2f} & {rmse:6.2f} & {corr:6.2f} \\\\")

    CAODs=[]
    AOD = AODvgl.sel(model=models[idx]).mean(dim=['station','model'],skipna=True)
    AODseas = AOD.groupby('day.season').mean(skipna=True)
    for seas in ['DJF','MAM','JJA','SON']:
        CAODs.append(AODseas[f"AOD{wvl}"].sel(season=seas).values)
    CAODs.append(AOD[f"AOD{wvl}"].mean(dim='day',skipna=True).values)

    print("& T-CARS "+f"  & {CAODs[0]:6.2f} & {CAODs[1]:6.2f} & {CAODs[2]:6.2f}  & {CAODs[3]:6.2f}  & {CAODs[4]:6.2f} &-&-&-\\\\")
        
    print("\\middlehline")





\multirow{4}{*}{\rotatebox[origin=c]{90}{0\,\unit{nm}}}
& CEM           &   0.05 &   0.06 &   0.07  &   0.05  &   0.06 &   0.00 &   0.02 &   0.77 \\
& MMAC          &   0.11 &   0.15 &   0.20  &   0.10  &   0.15 &   0.10 &   0.12 &   0.77 \\
& METSTAT       &   0.03 &   0.04 &   0.07  &   0.04  &   0.05 &  -0.01 &   0.02 &   0.78 \\
& T-CARS   &   0.03 &   0.05 &   0.08  &   0.03  &   0.06 &-&-&-\\
\middlehline
\multirow{4}{*}{\rotatebox[origin=c]{90}{550\,\unit{nm}}}
& MRMv6.1       &   0.10 &   0.10 &   0.11  &   0.09  &   0.10 &  -0.04 &   0.08 &   0.50 \\
& ESRA          &   0.08 &   0.12 &   0.16  &   0.10  &   0.12 &  -0.01 &   0.06 &   0.68 \\
& Heliosat-1    &   0.04 &   0.07 &   0.12  &   0.05  &   0.07 &  -0.05 &   0.08 &   0.63 \\
& T-CARS   &   0.06 &   0.14 &   0.19  &   0.09  &   0.13 &-&-&-\\
\middlehline
\multirow{2}{*}{\rotatebox[origin=c]{90}{700\,\unit{nm}}}
& Solis simple  &   0.07 &   0.08 &   0.07  &   0.07  &   0.07 &  -0.02 &   0.05 &   0.44 \\
& T-CARS   &   0.

In [5]:
def get_metrics(O,P):
    #ensure no nans in data
    idx = ~np.isnan(O)*~np.isinf(O)
    idx*= ~np.isnan(P)*~np.isinf(P)
    O = O[idx]
    P = P[idx]
    # precalculation
    N = len(O)
    N1 = 1./float(N)
    N2 = 1./(float(N)+1.)
    DELTA = P-O
    SUM = P+O
    # MBE
    MBE = N1 * np.sum(DELTA)
    # RMSE
    RMSE = np.sqrt(N2*np.sum(DELTA**2))
    # R
    ccoef = np.corrcoef(P,O)
    ccoef = ccoef[0][1]
    return MBE,RMSE,ccoef

AODvgl=dsAOD.copy()
AODvgl = AODvgl.where(AODvgl.CAOD>0,drop=True)
AODvgl = AODvgl.where(AODvgl.CAOD<0.7,drop=True)
AODvgl = AODvgl.where(AODvgl.CAOD.round(3)!=0.3,drop=True)


## make Table
print("\\multirow{8}{*}{\\rotatebox[origin=c]{90}{550\\,\\unit{nm}}}") 
# for wvl in [0,550,700]:
#     idx = models_aodwvl==wvl
    

#     for i,model in enumerate(zip(models[idx],models_label[idx])):
for m,model in enumerate(zip(models,models_label)):
    wvl = models_aodwvl[m]
    CAODs=[]
    model,mlabel=model

    AOD = AODvgl.mean(dim=['station'],skipna=True).sel(model=model)
    if wvl == 0: 
        AOD.CAOD.values =(AOD.CAOD * (AOD.AOD550 / AOD.AOD0)).values
    if wvl == 700:
        AE = calc_AE(AOD.AOD550,AOD.AOD700,550,700)
        AOD.CAOD.values = calc_AOD(550.,AOD.CAOD.values,700,AE.values)#(AOD.CAOD * (AOD.AOD550 / AOD.AOD700)).values
    AODseas = AOD.groupby('day.season').mean(skipna=True)
    for seas in ['DJF','MAM','JJA','SON']:
        CAODs.append(AODseas.CAOD.sel(season=seas).values)
    CAODs.append(AOD.CAOD.mean(dim='day',skipna=True).values)
    EARE = AOD[f'AOD550'].values
    DARE = AOD.CAOD.values
    mbe,rmse,corr = get_metrics(EARE,DARE)   

    print(f"& {mlabel:12s}  & {CAODs[0]:6.2f} & {CAODs[1]:6.2f} & {CAODs[2]:6.2f}  & {CAODs[3]:6.2f}  & {CAODs[4]:6.2f} & {mbe:6.2f} & {rmse:6.2f} & {corr:6.2f} \\\\")

        
CAODs=[]
AOD = AODvgl.sel(model=models[idx]).mean(dim=['station','model'],skipna=True)
AODseas = AOD.groupby('day.season').mean(skipna=True)
for seas in ['DJF','MAM','JJA','SON']:
    CAODs.append(AODseas[f"AOD550"].sel(season=seas).values)
CAODs.append(AOD[f"AOD550"].mean(dim='day',skipna=True).values)

print("& T-CARS "+f"  & {CAODs[0]:6.2f} & {CAODs[1]:6.2f} & {CAODs[2]:6.2f}  & {CAODs[3]:6.2f}  & {CAODs[4]:6.2f} &-&-&-\\\\")

print("\\middlehline")





\multirow{8}{*}{\rotatebox[origin=c]{90}{550\,\unit{nm}}}
& MRMv6.1       &   0.10 &   0.10 &   0.11  &   0.09  &   0.10 &  -0.04 &   0.08 &   0.50 \\
& ESRA          &   0.08 &   0.12 &   0.16  &   0.10  &   0.12 &  -0.01 &   0.06 &   0.68 \\
& Heliosat-1    &   0.04 &   0.07 &   0.12  &   0.05  &   0.07 &  -0.05 &   0.08 &   0.63 \\
& Solis simple  &   0.09 &   0.11 &   0.10  &   0.09  &   0.10 &  -0.03 &   0.08 &   0.48 \\
& CEM           &   0.13 &   0.14 &   0.16  &   0.13  &   0.15 &   0.01 &   0.06 &   0.55 \\
& MMAC          &   0.28 &   0.35 &   0.50  &   0.27  &   0.38 &   0.25 &   0.29 &   0.71 \\
& METSTAT       &   0.09 &   0.10 &   0.19  &   0.09  &   0.13 &  -0.01 &   0.06 &   0.74 \\
& T-CARS   &   0.06 &   0.14 &   0.20  &   0.09  &   0.13 &-&-&-\\
\middlehline
